In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data

In [4]:
X_train, X_val, X_test, vocab = ptb_raw_data('bigdata/simple-examples/data')

In [6]:
# Shoutout to: 
# https://stackoverflow.com/questions/41695893/tensorflow-conditionally-add-variable-scope
class empty_scope():
 def __init__(self):
     pass
 def __enter__(self):
     pass
 def __exit__(self, type, value, traceback):
     pass

def cond_name_scope(scope):
    return empty_scope() if scope is None else tf.name_scope(scope)

def cond_variable_scope(scope):
    return empty_scope() if scope is None else tf.variable_scope(scope)

In [5]:
def ptb_batcher(raw_data, batch_size, num_steps):
    """Return a batch of data.

    Equivalent of ptb_producer that I wrote to understand all the TF concepts.
    """
    with tf.name_scope('batcher'):
        tf_raw_data = tf.convert_to_tensor(
            raw_data,
            name='raw_data',
            dtype=tf.int32
        )
        data_len = tf.size(
            tf_raw_data,
            name='num_elems'
        )
        num_batches = tf.floordiv(
            data_len, batch_size,
            name='num_batches'
        )
        data = tf.reshape(
            tf_raw_data[:batch_size * num_batches],
            [batch_size, num_batches],
            name='data'
        )
        batches_per_epoch = tf.floordiv(
            num_batches - 1, num_steps,
            name='batches_per_epoch'
        )
        tf_queue = tf.train.range_input_producer(
            limit=batches_per_epoch, shuffle=False
        )
        i = tf_queue.dequeue(name='iter_idx')
        x = tf.identity(
            data[:, (i * num_steps):((i+1) * num_steps)], 
            name='x'
        )
        x.set_shape([batch_size, num_steps])
        y = tf.identity(
            data[:, (1 + i * num_steps):(1 + (i+1) * num_steps)],
            name='y'
        )
        y.set_shape([batch_size, num_steps])
        return x, y
        

In [7]:
def gru_update(x_t, h_t_minus_1, gru_params, timestep=None):
    U_z, W_z, b_z = gru_params['U_z'], gru_params['W_z'], gru_params['b_z']
    U_r, W_r, b_r = gru_params['U_r'], gru_params['W_r'], gru_params['b_r']
    U_h, W_h, b_h = gru_params['U_h'], gru_params['W_h'], gru_params['b_h']
    with tf.name_scope('gru_calculations'):
        with cond_name_scope(timestep):
            r_t = tf.sigmoid(
                tf.matmul(x_t, W_r) + tf.matmul(h_t_minus_1, U_r) + b_r,
                name='r' + (
                    '_{0}'.format(timestep) if timestep is not None else ''
                ),
            )
            z_t = tf.sigmoid(
                tf.matmul(x_t, W_z) + tf.matmul(h_t_minus_1, U_z) + b_z,
                name='z' + (
                    '_{0}'.format(timestep) if timestep is not None else ''
                ),
            )
            h_tilde_t = tf.tanh(
                tf.matmul(x_t, W_h) + tf.matmul(h_t_minus_1 * r_t, U_h) + b_h,
                name='h_tilde' + (
                    '_{0}'.format(timestep) if timestep is not None else ''
                ),
            )
            h_t = z_t * h_t_minus_1 + (1 - z_t) * h_tilde_t
    return h_t    

In [108]:
def make_rnn(vocab_size, embedding_size, hidden_size, batch_size, num_steps):
    """Simplified version of PTBModel"""
    with tf.variable_scope(
        'RNNParams',
        reuse=False,
        initializer=tf.random_uniform_initializer(-0.05, 0.05),
    ):
        embedding_matrix = tf.get_variable(
            'embedding',
            [vocab_size, embedding_size],
            dtype=tf.float16,
        )
        gru_params = {
            'U_z': tf.get_variable(
                'U_z',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'W_z': tf.get_variable(
                'W_z',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'b_z': tf.get_variable(
                'b_z',
                [hidden_size],
                dtype=tf.float16,
            ),
            'U_r': tf.get_variable(
                'U_r',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'W_r': tf.get_variable(
                'W_r',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'b_r': tf.get_variable(
                'b_r',
                [hidden_size],
                dtype=tf.float16,
            ),
            'U_h': tf.get_variable(
                'U_h',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'W_h': tf.get_variable(
                'W_h',
                [embedding_size, hidden_size],
                dtype=tf.float16,
            ),
            'b_h': tf.get_variable(
                'b_h',
                [hidden_size],
                dtype=tf.float16,
            ),
        }
        softmax_w = tf.get_variable(
            'softmax_w',
            [hidden_size, vocab_size],
            dtype=tf.float16,
        )
        softmax_b = tf.get_variable(
            'softmax_b',
            [vocab_size]
        )

    with tf.name_scope('RNN'):
        input_sequence = tf.placeholder(
            tf.int32,
            shape=[batch_size, num_steps]
        )
        embedded_inputs = tf.nn.embedding_lookup(
            embedding_matrix,
            input_sequence
        )
        

        
    return embedding_matrix, softmax_w, softmax_b, cell

In [110]:
tf.reset_default_graph()
embed = make_rnn(50, 3, 4, 5)
print(embed)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(embed))

(<tf.Variable 'Model/embedding:0' shape=(50, 3) dtype=float16_ref>, <tf.Variable 'Model/softmax_w:0' shape=(3, 50) dtype=float16_ref>, <tf.Variable 'Model/softmax_b:0' shape=(50,) dtype=float32_ref>, <tensorflow.contrib.rnn.python.ops.lstm_ops.LSTMBlockCell object at 0x118d610b8>)


TypeError: Fetch argument <tensorflow.contrib.rnn.python.ops.lstm_ops.LSTMBlockCell object at 0x118d610b8> has invalid type <class 'tensorflow.contrib.rnn.python.ops.lstm_ops.LSTMBlockCell'>, must be a string or Tensor. (Can not convert a LSTMBlockCell into a Tensor or Operation.)

In [100]:
tf_x, tf_y = ptb_batcher(np.arange(200), 4, 3)
print(tf_x, tf_y)sv = tf.train.Supervisor(logdir='logs')
with sv.managed_session() as sess:
    for i in range(2):
        xout, yout = sess.run([tf_x, tf_y])
        print(xout)
        print(yout)

Tensor("batcher/x:0", shape=(4, 3), dtype=int32) Tensor("batcher/y:0", shape=(4, 3), dtype=int32)
INFO:tensorflow:Starting standard services.


INFO:tensorflow:Starting standard services.


INFO:tensorflow:Starting queue runners.


INFO:tensorflow:Starting queue runners.


INFO:tensorflow:Recording summary at step None.


INFO:tensorflow:Recording summary at step None.


[[  0   1   2]
 [ 50  51  52]
 [100 101 102]
 [150 151 152]]
[[  1   2   3]
 [ 51  52  53]
 [101 102 103]
 [151 152 153]]
[[  3   4   5]
 [ 53  54  55]
 [103 104 105]
 [153 154 155]]
[[  4   5   6]
 [ 54  55  56]
 [104 105 106]
 [154 155 156]]


In [ ]:
with tf.name_scope('foo'):
    with tf.name_scope('foo')